---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [ ]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

Let's match these formats:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* 6/2008; 12/2009
* 2008; 2009

In [ ]:
#tmp1 = df.str.extractall(r'(?P<date>(?P<month>\d{1,2})*[-/]*(?P<day>\d{1,2})*[-/]*(?P<year>[1-2]\d{3}))')
tmp1 = df.str.extractall(r'(?P<date>(?P<month>\d{1,2})[-/](?P<day>\d{1,2})[-/](?P<year>\d{2,4}))')
tmp1.head()

Now the second format:
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;

In [ ]:
tmp2 = df.str.extractall(r'(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ,\.]{1,2}(?P<day>\d{1,2})[ ,\.]{1,2}(?P<year>\d{2,4}))')
tmp2.head()

The third format:
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009

In [ ]:
tmp3 = df.str.extractall(r'(?P<date>(?P<day>\d{2})[ ,\.-]+(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ , \.-]+(?P<year>\d{2,4}))')
tmp3.head()

Now the fourth format:
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009



In [ ]:
tmp4 = df.str.extractall(r'(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (?P<day>\d{1,2})[thsnd]{2}, (?P<year>\d{4}))')
tmp4.head()

Now the fifth format:
* Feb 2009; Sep 2009; Oct 2010

In [ ]:
tmp5 = df.str.extractall(r'(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ,](?P<year>\d{4}))')
tmp5.head()

Now the sixth format:
* 6/2008; 12/2009

In [ ]:
tmp6 = df.str.extractall(r'(?P<date>(?P<month>\d{1,2})/(?P<year>\d{4}))')
tmp6.head()

And finally the last format:
* 2008, 2009.

In [ ]:
tmp7 = df.str.extractall(r'(?P<date>(?P<year>[ ]*[1-2]\d{3}[ ]*))')
tmp7.head()

Let's check how many matches we have

In [ ]:
tmplist = [tmp1, tmp2, tmp3, tmp4, tmp5, tmp6, tmp7]
num_matches = 0
for tmp in tmplist:
    num_matches += tmp.shape[0]
print(num_matches)

They are more than the number of rows in the original data. We have some overlapping (especially from the 7th format...).
We will have to fix this.

We now need to prepare the data in order to be put together and sorted.
Let's start by mapping month names to month numbers

In [ ]:
months_label = {'Jan': '01',
                'Feb': '02',
                'Mar': '03',
                'Apr': '04',
                'May': '05',
                'Jun': '06',
                'Jul': '07',
                'Aug': '08',
                'Sep': '09',
                'Oct': '10',
                'Nov': '11',
                'Dec': '12'
}
tmp2['month'] = tmp2['month'].map(months_label)
tmp3['month'] = tmp3['month'].map(months_label)
tmp4['month'] = tmp4['month'].map(months_label)
tmp5['month'] = tmp5['month'].map(months_label)

We need to add the missing 'day' and 'month' columns to some of the dataframes

In [ ]:
tmp5['day'] = tmp6['day'] = tmp7['day'] = np.nan
#tmp5['day'] = np.nan
tmp7['month'] = np.nan

We now have to concatenate all these dataframes together...

In [ ]:
tmplist = [tmp2, tmp3, tmp4, tmp5, tmp6, tmp7]
#tmplist = [tmp2, tmp3, tmp4, tmp5]
final = tmp1
for tmp in tmplist:
    final = pd.concat([final, tmp[['date', 'year', 'month', 'day']]], axis = 0)
final.reset_index(inplace=True)
final = pd.DataFrame(final[['level_0', 'date', 'day', 'month', 'year']])
final.set_index('level_0', inplace=True)
final.index.name = 'index'
final['num_missing_info'] = final.isna().sum(axis=1)

In [ ]:
final.head()

We need to keep fixing our final dataset:
1. When we have multiple matches per index we keep the one with least number of missing values

In [ ]:
final = final.merge(final
                    .groupby(final.index)['num_missing_info']
                    .min()
                    .rename('min_missing')
                    .to_frame(),
                    left_index = True,
                    right_index = True
                   )
final = final[final['num_missing_info'] == final['min_missing']]

3. The df is 501 now...we have a duplicate. It happened to be a wrong match where the year is 3 digits. Let's take care of this

In [ ]:
final = final[final['year'].str.len()!=3]

2. Fill missing month/day with 01

In [ ]:
#final.replace('', '01', inplace=True)
final.fillna(value='01', inplace=True)

2. Pad single digit days/months with a 0 on the left

In [ ]:
final['day'] = final['day'].str.zfill(2)
final['month'] = final['month'].str.zfill(2)
final['year'] = final['year'].str.rjust(width = 3, fillchar = '9').str.rjust(width = 4, fillchar = '1')


Let's create the yyyymmdd column

In [ ]:
final_out = (final['year']+final['month']+final['day']).str.replace(' ', '').astype(int)

Let's get this ready for the final output

In [ ]:
final_out.sort_values(inplace=True)


final_df = final_out.to_frame()
final_df = final_df.merge(df.to_frame(),
                          left_index = True,
                          right_index = True
                         ).reset_index()
final_df.columns = ['index', 'date', 'original_string']

Let's put everything into the function to submit. Note: this function is not compatible with the autograder, as there is a bug in the version of pandas the autograter is using. The other file (Assignment+1+tosubmit) should be compatible with the autograder.

In [1]:
def date_sorter():
    
    # Load the data
    import pandas as pd

    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)
    
    # Match the various Date formats
    tmp1 = df.str.extractall(r'(?P<date>(?P<month>\d{1,2})[-/](?P<day>\d{1,2})[-/](?P<year>\d{2,4}))')
    tmp2 = df.str.extractall(r'(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ,\.]{1,2}(?P<day>\d{1,2})[ ,\.]{1,2}(?P<year>\d{2,4}))')
    tmp3 = df.str.extractall(r'(?P<date>(?P<day>\d{2})[ ,\.-]+(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ , \.-]+(?P<year>\d{2,4}))')
    tmp4 = df.str.extractall(r'(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) (?P<day>\d{1,2})[thsnd]{2}, (?P<year>\d{4}))')
    tmp5 = df.str.extractall(r'(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ,](?P<year>\d{4}))')
    tmp6 = df.str.extractall(r'(?P<date>(?P<month>\d{1,2})/(?P<year>\d{4}))')
    tmp7 = df.str.extractall(r'(?P<date>(?P<year>[ ]*[1-2]\d{3}[ ]*))')
    
    # map the months into integers
    months_label = {'Jan': '01',
                    'Feb': '02',
                    'Mar': '03',
                    'Apr': '04',
                    'May': '05',
                    'Jun': '06',
                    'Jul': '07',
                    'Aug': '08',
                    'Sep': '09',
                    'Oct': '10',
                    'Nov': '11',
                    'Dec': '12'
    }
    tmp2['month'] = tmp2['month'].map(months_label)
    tmp3['month'] = tmp3['month'].map(months_label)
    tmp4['month'] = tmp4['month'].map(months_label)
    tmp5['month'] = tmp5['month'].map(months_label)
    
    # Add the missing columns to some of the data.frames
    tmp5['day'] = tmp6['day'] = tmp7['day'] = np.nan
    tmp7['month'] = np.nan
    
    tmplist = [tmp2, tmp3, tmp4, tmp5, tmp6, tmp7]
    final = tmp1
    for tmp in tmplist:
        final = pd.concat([final, tmp[['date', 'year', 'month', 'day']]], axis = 0)
    final.reset_index(inplace=True)
    final = pd.DataFrame(final[['level_0', 'date', 'day', 'month', 'year']])
    final.set_index('level_0', inplace=True)
    final.index.name = 'index'
    final['num_missing_info'] = final.isna().sum(axis=1)    
    
    final = final.merge(final
                        .groupby(final.index)['num_missing_info']
                        .min()
                        .rename('min_missing')
                        .to_frame(),
                        left_index = True,
                        right_index = True
                       )
    final = final[final['num_missing_info'] == final['min_missing']]
    
    # We have a duplicate. It's a year with 3 digits (wrong match).
    # Let's get rid of it
    final = final[final['year'].str.len() != 3]
    
    # Fill gaps
    final.replace('', '01', inplace=True)
    final.fillna(value='01', inplace=True)
    
    # Pad month, day, and year to get the proper string lengths
    final['day'] = final['day'].str.zfill(2)
    final['month'] = final['month'].str.zfill(2)
    final['year'] = final['year'].str.rjust(width = 3, fillchar = '9').str.rjust(width = 4, fillchar = '1')
    
    # Let's prepare the output
    final_out = (final['year']+final['month']+final['day']).str.replace(' ', '').astype(int)
    final_out.sort_values(inplace=True)
    
    
    final_df = final_out.to_frame()
    final_df = final_df.merge(df.to_frame(),
                              left_index = True,
                              right_index = True
                             ).reset_index()
    final_df.columns = ['index', 'date', 'original_string']
    return final_df#final_df['index'].rename(None)

In [2]:
tmp_out = date_sorter()

/Applications/anaconda3/envs/testingenv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [3]:
tmp_out

,index,date,original_string
0,9,19710410,(4/10/71)Score-1Audit C Score Current:\n
1,84,19710518,5/18/71 Total time of visit (in minutes):\n
2,2,19710708,sshe plans to move as of 7/8/71 In-Home Servic...
3,53,19710711,7/11/71 SOS-10 Total Score:\n
4,28,19710912,9/12/71 [report_end]\n
5,474,19720101,sPatient reported losing three friends that pa...
6,153,19720113,13 Jan 1972 Primary Care Doctor:\n
7,13,19720126,1/26/72 Communication with referring physician...
8,129,19720506,06 May 1972 SOS-10 Total Score:\n
9,98,19720513,5/13/72 Other Adult Mental Health Outcomes Sca...


In [4]:
tmp_out.to_csv('assignment1_out.csv')

In [7]:
tmp_out.loc[43]['original_string']

'10 Oct 1974 Primary Care Doctor:\n'

In [9]:
tmp_out[tmp_out['index']==95]

,index,date,original_string
33,95,19740411,4/11/1974 Chief Complaint / HPI Chief Complain...
